In [1]:
import pandas as pd
import geopandas as gp
import numpy as np 
import os
import pdv_functions as pdv

## Election Results

Election results from Nevada Secretary of State (https://www.nvsos.gov/sos/elections/election-information/precinct-level-results).

## Load the SOS Election Results File

In [2]:
nv_22 = pd.read_csv("./raw-from-source/2022 General Election Results by Precinct.csv", index_col=False,encoding= 'unicode_escape')
nv_22.columns = nv_22.iloc[8]
nv_22 = nv_22[9:]
nv_22 = nv_22[[i for i in nv_22.columns if i is not None]]
nv_22 = nv_22[['Jurisdiction','Precinct ','Contest ','Selection','Votes']]
nv_22.columns = ['Jurisdiction','Precinct','Contest','Selection','Votes']

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/1167370931.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  nv_22 = pd.read_csv("./raw-from-source/2022 General Election Results by Precinct.csv", index_col=False,encoding= 'unicode_escape')


In [3]:
nv_22 = nv_22[~nv_22["Contest"].isna()]

In [4]:
contest_list = list(nv_22["Contest"].unique())

In [6]:
nv_2022_contest_names = ['United States Senator','Attorney General',
 'Governor','Lieutenant Governor','Secretary of State',
 'State Controller','State Treasurer'] + [i for i in contest_list if "U.S. Representative in Congress" in i] +[i for i in contest_list if "Court of Appeals" in i] +[i for i in contest_list if "State Senate" in i] + [i for i in contest_list if "State Assembly" in i] + [i for i in contest_list if "Supreme Court" in i] + [i for i in contest_list if "STATE QUESTION" in i]

In [7]:
nv_22 = nv_22[nv_22["Contest"].isin(nv_2022_contest_names)]

In [8]:
nv_22[nv_22["Votes"]=="*"]

,Jurisdiction,Precinct,Contest,Selection,Votes
1737,Carson City,996,Attorney General,"Chattah, Sigal",*
1738,Carson City,996,Attorney General,"Ford, Aaron D.",*
1739,Carson City,996,Attorney General,None Of These Candidates,*
1746,Carson City,996,"Court of Appeals Judge, Department 1","Forsberg, Rhonda K.",*
1747,Carson City,996,"Court of Appeals Judge, Department 1",None Of These Candidates,*
...,...,...,...,...,...
146177,Washoe,SPARKS 9559,United States Senator,"Laxalt, Adam Paul",*
146178,Washoe,SPARKS 9559,United States Senator,"Lindemann, Barry",*
146179,Washoe,SPARKS 9559,United States Senator,None Of These Candidates,*
146180,Washoe,SPARKS 9559,United States Senator,"RUBINSON, BARRY",*


In [9]:
nv_22["pivot_col"] = nv_22["Jurisdiction"]+"-"+nv_22["Precinct"]

In [10]:
nv_22["cand_col"] = nv_22["Contest"]+"-:-"+nv_22["Selection"]

In [11]:
nv_22["cand_col"].value_counts()

United States Senator-:-Scott, Neil                           1691
United States Senator-:-RUBINSON, BARRY                       1691
United States Senator-:-None Of These Candidates              1691
United States Senator-:-Lindemann, Barry                      1691
United States Senator-:-Laxalt, Adam Paul                     1691
                                                              ... 
State Assembly, District 11-:-Duran, Beatrice "Bea"             20
State Assembly, District  8-:-Nguyen, Duy                       19
State Assembly, District  8-:-Logan, Jenann                     19
State Assembly, District  7-:-Miller, Cameron Homer "C.H."      17
State Assembly, District  7-:-Palmer, Anthony "Tony"            17
Name: cand_col, Length: 173, dtype: int64

In [12]:
pivoted = pd.pivot_table(nv_22, index = ["pivot_col","Jurisdiction","Precinct"], columns = ["cand_col"],values = "Votes", aggfunc = sum )

In [13]:
pivoted.reset_index(inplace = True, drop = False)

In [14]:
pivoted

cand_col,pivot_col,Jurisdiction,Precinct,"Attorney General-:-Chattah, Sigal","Attorney General-:-Ford, Aaron D.",Attorney General-:-None Of These Candidates,"Court of Appeals Judge, Department 1-:-Forsberg, Rhonda K.","Court of Appeals Judge, Department 1-:-None Of These Candidates","Court of Appeals Judge, Department 1-:-Westbrook, Deborah","Court of Appeals Judge, Department 2-:-Gibbons, Michael",...,"U.S. Representative in Congress, District 3-:-Becker, April","U.S. Representative in Congress, District 3-:-Lee, Susie","U.S. Representative in Congress, District 4-:-Horsford, Steven A.","U.S. Representative in Congress, District 4-:-Peters, Sam","United States Senator-:-Cortez Masto, Catherine","United States Senator-:-Laxalt, Adam Paul","United States Senator-:-Lindemann, Barry",United States Senator-:-None Of These Candidates,"United States Senator-:-RUBINSON, BARRY","United States Senator-:-Scott, Neil"
0,Carson City-101,Carson City,101,263,348,29,156,153,306,452,...,NaN,NaN,NaN,NaN,340,300,2,11,11,5
1,Carson City-103,Carson City,103,402,515,35,224,185,491,721,...,NaN,NaN,NaN,NaN,487,472,4,6,6,8
2,Carson City-105,Carson City,105,442,431,33,190,172,472,700,...,NaN,NaN,NaN,NaN,404,523,4,6,0,4
3,Carson City-107,Carson City,107,539,542,44,292,239,519,817,...,NaN,NaN,NaN,NaN,482,656,3,19,7,5
4,Carson City-109,Carson City,109,555,625,59,268,288,644,925,...,NaN,NaN,NaN,NaN,570,683,13,16,8,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1686,White Pine-Precinct 5,White Pine,Precinct 5,716,144,36,189,290,404,650,...,NaN,NaN,NaN,NaN,145,757,9,11,7,13
1687,White Pine-Precinct 6,White Pine,Precinct 6,257,91,19,101,118,139,250,...,NaN,NaN,NaN,NaN,77,290,8,8,3,0
1688,White Pine-Precinct 7,White Pine,Precinct 7,92,29,2,30,29,62,91,...,NaN,NaN,NaN,NaN,24,100,1,1,3,1
1689,White Pine-Precinct 8,White Pine,Precinct 8,197,23,13,50,64,100,167,...,NaN,NaN,NaN,NaN,11,217,2,8,4,0


In [15]:
def get_election_type(contest):
    return "G"
#     first = contest.split("-:-")[0]
#     if first == 'Election':
#         e_type = "G"
#     elif first == "District" or first == "Dist":
#         e_type = "S"
#     elif first == 'Runoff':
#         e_type = "R"
#     elif first == 'Primary':
#         e_type = "P"
#     else:
#         print(contest)
#         raise ValueError
#     return e_type

def get_race(contest):
    mod_level = ""
    level = contest.split("-:-")[0]
    level_change_dict = {
        'Attorney General':'ATG',
        'Court of Appeals Judge':'CAJ',
        'Lieutenant Governor':'LTG',
        'Governor':'GOV',
        'State Controller':'CNT',
        'STATE QUESTION':'SQ',
        'Secretary of State':'SOS',
        'State Assembly':'SL',
        'State Senate':'SU',
        'President':'PRE',
        'United States Senator':'USS',
        
        'State Treasurer':'TRE',
        'Supreme Court Justice':'SCJ',
        'U.S. Representative in Congress':'CON'}
    for val in level_change_dict.keys():
        if val in level:
            mod_level = level_change_dict[val]
            break
    if mod_level == "":
        print("NO CONTEST", contest)
    if mod_level in ["CON", "SU", "SL","CAJ","SCJ"]:
            dist_num = str(level.split(" ")[-1])
            if mod_level == "SL":
                fill = 3
            elif mod_level in ["CON","SU"]:
                fill = 2
            else:
                fill = 1
            mod_level += str(dist_num).zfill(fill)
    elif mod_level in ["SQ"]:
        mod_level += str(level.split(" ")[3])
        mod_level = "22"+mod_level
    else:
        mod_level = "22"+mod_level
    return mod_level

def get_party(contest):
    return "X"
    
def get_name(contest):
    candidate = contest.split("-:-")[1]
    if candidate == "None Of These Candidates":
        return "WRI"
    candidate = candidate.upper()
    candidate = candidate.replace(" ", "")
    likely_last = candidate.split(" ")[-1]
    
    return likely_last[0:3]

In [16]:
keep_names  = ['pivot_col', 'Jurisdiction', 'Precinct']

contest_name_change_dict = {}

duplicate_value_list = []
for contest in pivoted.columns:
    if contest not in keep_names:
        # Add in a condition about the 20
        value = get_election_type(contest) + get_race(contest)+ get_party(contest)+ get_name(contest)
        if value in contest_name_change_dict.values():
            duplicate_value_list.append(value)
        contest_name_change_dict[contest] = value 
#         else:
#             value = get_election_type(contest) + "22" + get_race(contest)+ get_party(contest)+ get_name(contest)
#             if value in contest_name_change_dict.values():
#                 duplicate_value_list.append(value)
#             contest_name_change_dict[contest] = value 

contest_name_change_dict["State Assembly, District 28-:-D'Silva, Reuben"] = "GSL028XDSI"
contest_name_change_dict['State Assembly, District 40-:-O\'Neill, Philip "PK"'] = "GSL040XONE"

if len(duplicate_value_list) > 0:
    print("DUPLICATE VALUES")
    for val in [(k,v) for k, v in contest_name_change_dict.items() if v in duplicate_value_list]:
        print(val)

In [17]:
updated_dict = {'G22ATGXCHA': 'G22ATGRCHA',
 'G22ATGXFOR': 'G22ATGDFOR',
 'G22ATGXWRI': 'G22ATGOWRI',
 'GCAJ1XFOR': 'GCAJ1NFOR',
 'GCAJ1XWRI': 'GCAJ1NWRI',
 'GCAJ1XWES': 'GCAJ1NWES',
 'GCAJ2XGIB': 'GCAJ2NGIB',
 'GCAJ2XWRI': 'GCAJ2NWRI',
 'GCAJ3XBUL': 'GCAJ3NBUL',
 'GCAJ3XWRI': 'GCAJ3NWRI',
 'G22GOVXBRI': 'G22GOVABRI',
 'G22GOVXDAV': 'G22GOVLDAV',
 'G22GOVXLOM': 'G22GOVRLOM',
 'G22GOVXWRI': 'G22GOVOWRI',
 'G22GOVXSIS': 'G22GOVDSIS',
 'G22LTGXANT': 'G22LTGRANT',
 'G22LTGXCAN': 'G22LTGDCAN',
 'G22LTGXDEL': 'G22LTGIDEL',
 'G22LTGXHOG': 'G22LTGAHOG',
 'G22LTGXWRI': 'G22LTGOWRI',
 'G22LTGXTAC': 'G22LTGLTAC',
 'G22SQ1XNO': 'G22SQ1NO',
 'G22SQ1XYES': 'G22SQ1YES',
 'G22SQ2XNO': 'G22SQ2NO',
 'G22SQ2XYES': 'G22SQ2YES',
 'G22SQ3XNO': 'G22SQ3NO',
 'G22SQ3XYES': 'G22SQ3YES',
 'G22SOSXAGU': 'G22SOSDAGU',
 'G22SOSXCRA': 'G22SOSLCRA',
 'G22SOSXHAN': 'G22SOSAHAN',
 'G22SOSXMAR': 'G22SOSRMAR',
 'G22SOSXWRI': 'G22SOSOWRI',
 'GSL001XBRI': 'GSL001RBRI',
 'GSL001XMCA': 'GSL001IMCA',
 'GSL001XMON': 'GSL001DMON',
 'GSL002XBED': 'GSL002LBED',
 'GSL002XCHR': 'GSL002DCHR',
 'GSL002XKAS': 'GSL002RKAS',
 'GSL003XLEM': 'GSL003RLEM',
 'GSL003XTOR': 'GSL003DTOR',
 'GSL004XBUR': 'GSL004LBUR',
 'GSL004XMCA': 'GSL004RMCA',
 'GSL005XMIL': 'GSL005DMIL',
 'GSL005XMOR': 'GSL005LMOR',
 'GSL005XQUI': 'GSL005RQUI',
 'GSL006XRIO': 'GSL006RRIO',
 'GSL006XSUM': 'GSL006DSUM',
 'GSL007XMIL': 'GSL007DMIL',
 'GSL007XPAL': 'GSL007RPAL',
 'GSL008XLOG': 'GSL008RLOG',
 'GSL008XNGU': 'GSL008DNGU',
 'GSL009XFLE': 'GSL009RFLE',
 'GSL009XYEA': 'GSL009DYEA',
 'GSL010XHER': 'GSL010RHER',
 'GSL010XNGU': 'GSL010DNGU',
 'GSL011XDUR': 'GSL011DDUR',
 'GSL011XKRA': 'GSL011RKRA',
 'GSL012XCAR': 'GSL012DCAR',
 'GSL012XLAR': 'GSL012RLAR',
 'GSL013XHIB': 'GSL013RHIB',
 'GSL013XRUC': 'GSL013DRUC',
 'GSL014XMOS': 'GSL014DMOS',
 'GSL014XSTA': 'GSL014RSTA',
 'GSL015XBAN': 'GSL015RBAN',
 'GSL015XWAT': 'GSL015DWAT',
 'GSL016XGON': 'GSL016DGON',
 'GSL016XHOL': 'GSL016RHOL',
 'GSL017XPAW': 'GSL017RPAW',
 'GSL017XTHO': 'GSL017DTHO',
 'GSL018XCON': 'GSL018DCON',
 'GSL018XDEC': 'GSL018RDEC',
 'GSL019XYUR': 'GSL019RYUR',
 'GSL020XLAR': 'GSL020LLAR',
 'GSL020XORE': 'GSL020DORE',
 'GSL020XVAU': 'GSL020RVAU',
 'GSL021XMAR': 'GSL021DMAR',
 'GSL021XPET': 'GSL021RPET',
 'GSL022XHAR': 'GSL022RHAR',
 'GSL022XRAM': 'GSL022DRAM',
 'GSL023XBRI': 'GSL023DBRI',
 'GSL023XGAL': 'GSL023RGAL',
 'GSL023XMAN': 'GSL023LMAN',
 'GSL024XKIN': 'GSL024RKIN',
 'GSL024XPET': 'GSL024DPET',
 'GSL025XKUM': 'GSL025RKUM',
 'GSL025XLAR': 'GSL025DLAR',
 'GSL026XDEL': 'GSL026RDEL',
 'GSL026XMIT': 'GSL026LMIT',
 'GSL027XORT': 'GSL027RORT',
 'GSL027XTAY': 'GSL027DTAY',
 'GSL028XBRO': 'GSL028RBRO',
 'GSL028XDSI': 'GSL028DDSI',
 'GSL029XCOH': 'GSL029DCOH',
 'GSL029XKNI': 'GSL029RKNI',
 'GSL030XAND': 'GSL030DAND',
 'GSL030XMCG': 'GSL030LMCG',
 'GSL030XROD': 'GSL030RROD',
 'GSL031XDIC': 'GSL031RDIC',
 'GSL032XHAN': 'GSL032RHAN',
 'GSL033XGAR': 'GSL033DGAR',
 'GSL033XGUR': 'GSL033RGUR',
 'GSL034XBIL': 'GSL034DBIL',
 'GSL034XBUT': 'GSL034RBUT',
 'GSL035XGOR': 'GSL035DGOR',
 'GSL035XJON': 'GSL035RJON',
 'GSL035XROB': 'GSL035LROB',
 'GSL036XHAF': 'GSL036RHAF',
 'GSL037XBAC': 'GSL037DBAC',
 'GSL037XDEA': 'GSL037RDEA',
 'GSL037XTED': 'GSL037LTED',
 'GSL038XKOE': 'GSL038RKOE',
 'GSL039XGRA': 'GSL039RGRA',
 'GSL039XNOB': 'GSL039DNOB',
 'GSL040XMCD': 'GSL040DMCD',
 'GSL040XONE': 'GSL040RONE',
 'GSL040XTOL': 'GSL040LTOL',
 'GSL041XBOD': 'GSL041RBOD',
 'GSL041XJAU': 'GSL041DJAU',
 'GSL041XMCN': 'GSL041LMCN',
 'GSL042XBRO': 'GSL042DBRO',
 'GSL042XFAC': 'GSL042RFAC',
 'G22CNTXMAT': 'G22CNTRMAT',
 'G22CNTXWRI': 'G22CNTOWRI',
 'G22CNTXPRO': 'G22CNTLPRO',
 'G22CNTXSPI': 'G22CNTDSPI',
 'GSU10XCUN': 'GSU10LCUN',
 'GSU10XDON': 'GSU10DDON',
 'GSU10XGRA': 'GSU10RGRA',
 'GSU12XARR': 'GSU12RARR',
 'GSU12XPAZ': 'GSU12DPAZ',
 'GSU13XBUE': 'GSU13RBUE',
 'GSU13XDAL': 'GSU13DDAL',
 'GSU14XHAN': 'GSU14RHAN',
 'GSU16XKRA': 'GSU16RKRA',
 'GSU16XSIM': 'GSU16DSIM',
 'GSU17XTIT': 'GSU17RTIT',
 'GSU02XFLO': 'GSU02DFLO',
 'GSU02XHEN': 'GSU02RHEN',
 'GSU20XFOU': 'GSU20DFOU',
 'GSU20XMIL': 'GSU20LMIL',
 'GSU20XSTO': 'GSU20RSTO',
 'GSU21XLAR': 'GSU21RLAR',
 'GSU21XOHR': 'GSU21DOHR',
 'GSU08XLOO': 'GSU08DLOO',
 'GSU08XPAU': 'GSU08RPAU',
 'GSU09XBRO': 'GSU09RBRO',
 'GSU09XSCH': 'GSU09DSCH',
 'G22TREXCON': 'G22TREDCON',
 'G22TREXELL': 'G22TRELELL',
 'G22TREXFIO': 'G22TRERFIO',
 'G22TREXHEN': 'G22TREAHEN',
 'G22TREXWRI': 'G22TREOWRI',
 'GSCJAXBEL': 'GSCJANBEL',
 'GSCJAXWRI': 'GSCJANWRI',
 'GSCJEXWRI': 'GSCJENWRI',
 'GSCJEXPAR': 'GSCJENPAR',
 'GCON01XCAV': 'GCON01LCAV',
 'GCON01XROB': 'GCON01RROB',
 'GCON01XTIT': 'GCON01DTIT',
 'GCON02XAMO': 'GCON02RAMO',
 'GCON02XBAB': 'GCON02LBAB',
 'GCON02XBES': 'GCON02ABES',
 'GCON02XKRA': 'GCON02DKRA',
 'GCON03XBEC': 'GCON03RBEC',
 'GCON03XLEE': 'GCON03DLEE',
 'GCON04XHOR': 'GCON04DHOR',
 'GCON04XPET': 'GCON04RPET',
 'G22USSXCOR': 'G22USSDCOR',
 'G22USSXLAX': 'G22USSRLAX',
 'G22USSXLIN': 'G22USSILIN',
 'G22USSXWRI': 'G22USSOWRI',
 'G22USSXRUB': 'G22USSARUB',
 'G22USSXSCO': 'G22USSLSCO'}

In [18]:
reversed_contest_name_change_dict = {v:k for k,v in contest_name_change_dict.items()}

In [19]:
reversed_contest_name_change_dict

{'G22ATGXCHA': 'Attorney General-:-Chattah, Sigal',
 'G22ATGXFOR': 'Attorney General-:-Ford, Aaron D.',
 'G22ATGXWRI': 'Attorney General-:-None Of These Candidates',
 'GCAJ1XFOR': 'Court of Appeals Judge, Department 1-:-Forsberg, Rhonda K.',
 'GCAJ1XWRI': 'Court of Appeals Judge, Department 1-:-None Of These Candidates',
 'GCAJ1XWES': 'Court of Appeals Judge, Department 1-:-Westbrook, Deborah',
 'GCAJ2XGIB': 'Court of Appeals Judge, Department 2-:-Gibbons, Michael',
 'GCAJ2XWRI': 'Court of Appeals Judge, Department 2-:-None Of These Candidates',
 'GCAJ3XBUL': 'Court of Appeals Judge, Department 3-:-Bulla, Bonnie',
 'GCAJ3XWRI': 'Court of Appeals Judge, Department 3-:-None Of These Candidates',
 'G22GOVXBRI': 'Governor-:-Bridges, Ed',
 'G22GOVXDAV': 'Governor-:-Davis, Brandon',
 'G22GOVXLOM': 'Governor-:-Lombardo, Joe',
 'G22GOVXWRI': 'Governor-:-None Of These Candidates',
 'G22GOVXSIS': 'Governor-:-SISOLAK, Steve',
 'G22LTGXANT': 'Lieutenant Governor-:-Anthony, Stavros',
 'G22LTGXCAN':

In [20]:
sorted_updated_dict = dict(sorted(updated_dict.items(), key=lambda x:x[1]))

In [21]:
print(list(sorted_updated_dict.values()))

['G22ATGDFOR', 'G22ATGOWRI', 'G22ATGRCHA', 'G22CNTDSPI', 'G22CNTLPRO', 'G22CNTOWRI', 'G22CNTRMAT', 'G22GOVABRI', 'G22GOVDSIS', 'G22GOVLDAV', 'G22GOVOWRI', 'G22GOVRLOM', 'G22LTGAHOG', 'G22LTGDCAN', 'G22LTGIDEL', 'G22LTGLTAC', 'G22LTGOWRI', 'G22LTGRANT', 'G22SOSAHAN', 'G22SOSDAGU', 'G22SOSLCRA', 'G22SOSOWRI', 'G22SOSRMAR', 'G22SQ1NO', 'G22SQ1YES', 'G22SQ2NO', 'G22SQ2YES', 'G22SQ3NO', 'G22SQ3YES', 'G22TREAHEN', 'G22TREDCON', 'G22TRELELL', 'G22TREOWRI', 'G22TRERFIO', 'G22USSARUB', 'G22USSDCOR', 'G22USSILIN', 'G22USSLSCO', 'G22USSOWRI', 'G22USSRLAX', 'GCAJ1NFOR', 'GCAJ1NWES', 'GCAJ1NWRI', 'GCAJ2NGIB', 'GCAJ2NWRI', 'GCAJ3NBUL', 'GCAJ3NWRI', 'GCON01DTIT', 'GCON01LCAV', 'GCON01RROB', 'GCON02ABES', 'GCON02DKRA', 'GCON02LBAB', 'GCON02RAMO', 'GCON03DLEE', 'GCON03RBEC', 'GCON04DHOR', 'GCON04RPET', 'GSCJANBEL', 'GSCJANWRI', 'GSCJENPAR', 'GSCJENWRI', 'GSL001DMON', 'GSL001IMCA', 'GSL001RBRI', 'GSL002DCHR', 'GSL002LBED', 'GSL002RKAS', 'GSL003DTOR', 'GSL003RLEM', 'GSL004LBUR', 'GSL004RMCA', 'GSL005DMIL

In [22]:
to_export_dict = {v: reversed_contest_name_change_dict[k] for k,v in sorted_updated_dict.items()}

In [23]:
holder = pd.DataFrame(to_export_dict.keys(),to_export_dict.values())
holder.to_csv("./field_names.csv", index = True)

In [24]:
pivoted.rename(columns = contest_name_change_dict, inplace = True)
pivoted.rename(columns = updated_dict, inplace = True)
pivoted.rename(columns = {"pivot_col":"UNIQUE_ID"}, inplace = True)
pivoted = pivoted.fillna(0)

# Add a COUNTYFP

In [25]:
#uploading national FIPS codes to apply back to file
FIPS = pd.read_csv('./raw-from-source/FIPS/US_FIPS_Codes.csv')
FIPS = FIPS[FIPS['State'] == 'Nevada']

FIPS_dict = dict(zip(FIPS["County Name"], FIPS["FIPS County"].astype(str).str.zfill(3)))


In [26]:
pivoted["COUNTYFP"] = pivoted["Jurisdiction"].map(FIPS_dict)

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/975467616.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pivoted["COUNTYFP"] = pivoted["Jurisdiction"].map(FIPS_dict)


## Identify Low Vote Precincts

In [27]:
pivoted_astericks = pivoted.copy(deep = True)
pivoted_astericks_unmod = pivoted.copy(deep = True)

In [28]:
for jurisdiction in list(nv_22[nv_22['Votes']=="*"]["Jurisdiction"].unique()):
    print(jurisdiction)
    print(nv_22[(nv_22['Votes']=="*")&(nv_22["Jurisdiction"]==jurisdiction)].shape)

Carson City
(51, 7)
Churchill
(4, 7)
Clark
(9362, 7)
Douglas
(58, 7)
Elko
(57, 7)
Esmeralda
(70, 7)
Eureka
(57, 7)
Lincoln
(4, 7)
Lyon
(55, 7)
Mineral
(55, 7)
Nye
(110, 7)
Storey
(480, 7)
Washoe
(11251, 7)


# Work Through the Results

In [29]:
pivoted

cand_col,UNIQUE_ID,Jurisdiction,Precinct,G22ATGRCHA,G22ATGDFOR,G22ATGOWRI,GCAJ1NFOR,GCAJ1NWRI,GCAJ1NWES,GCAJ2NGIB,...,GCON03DLEE,GCON04DHOR,GCON04RPET,G22USSDCOR,G22USSRLAX,G22USSILIN,G22USSOWRI,G22USSARUB,G22USSLSCO,COUNTYFP
0,Carson City-101,Carson City,101,263,348,29,156,153,306,452,...,0,0,0,340,300,2,11,11,5,510
1,Carson City-103,Carson City,103,402,515,35,224,185,491,721,...,0,0,0,487,472,4,6,6,8,510
2,Carson City-105,Carson City,105,442,431,33,190,172,472,700,...,0,0,0,404,523,4,6,0,4,510
3,Carson City-107,Carson City,107,539,542,44,292,239,519,817,...,0,0,0,482,656,3,19,7,5,510
4,Carson City-109,Carson City,109,555,625,59,268,288,644,925,...,0,0,0,570,683,13,16,8,6,510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1686,White Pine-Precinct 5,White Pine,Precinct 5,716,144,36,189,290,404,650,...,0,0,0,145,757,9,11,7,13,033
1687,White Pine-Precinct 6,White Pine,Precinct 6,257,91,19,101,118,139,250,...,0,0,0,77,290,8,8,3,0,033
1688,White Pine-Precinct 7,White Pine,Precinct 7,92,29,2,30,29,62,91,...,0,0,0,24,100,1,1,3,1,033
1689,White Pine-Precinct 8,White Pine,Precinct 8,197,23,13,50,64,100,167,...,0,0,0,11,217,2,8,4,0,033


In [30]:
pivoted[pivoted["UNIQUE_ID"]=='Carson City-996']

cand_col,UNIQUE_ID,Jurisdiction,Precinct,G22ATGRCHA,G22ATGDFOR,G22ATGOWRI,GCAJ1NFOR,GCAJ1NWRI,GCAJ1NWES,GCAJ2NGIB,...,GCON03DLEE,GCON04DHOR,GCON04RPET,G22USSDCOR,G22USSRLAX,G22USSILIN,G22USSOWRI,G22USSARUB,G22USSLSCO,COUNTYFP
24,Carson City-996,Carson City,996,*,*,*,*,*,*,*,...,0,0,0,*,*,*,*,*,*,510


In [31]:
list(nv_22[nv_22['Votes']=="*"]["pivot_col"].unique())

['Carson City-996',
 'Churchill-Precinct 12',
 'Churchill-Precinct 14',
 'Clark-1002',
 'Clark-1003',
 'Clark-1014',
 'Clark-1361',
 'Clark-1363',
 'Clark-1553',
 'Clark-1665',
 'Clark-1732',
 'Clark-2000',
 'Clark-2012',
 'Clark-2017',
 'Clark-2020',
 'Clark-2042',
 'Clark-2057',
 'Clark-2059',
 'Clark-2060',
 'Clark-2061',
 'Clark-2064',
 'Clark-2068',
 'Clark-2069',
 'Clark-2070',
 'Clark-2075',
 'Clark-2078',
 'Clark-2356',
 'Clark-2468',
 'Clark-2478',
 'Clark-2486',
 'Clark-2487',
 'Clark-2488',
 'Clark-2489',
 'Clark-2503',
 'Clark-2608',
 'Clark-2612',
 'Clark-2623',
 'Clark-2625',
 'Clark-2641',
 'Clark-2681',
 'Clark-2683',
 'Clark-2685',
 'Clark-2694',
 'Clark-2703',
 'Clark-2706',
 'Clark-2723',
 'Clark-2724',
 'Clark-2742',
 'Clark-3004',
 'Clark-3005',
 'Clark-3046',
 'Clark-3047',
 'Clark-3051',
 'Clark-3053',
 'Clark-3054',
 'Clark-3070',
 'Clark-3151',
 'Clark-3191',
 'Clark-3433',
 'Clark-3439',
 'Clark-3457',
 'Clark-3537',
 'Clark-3545',
 'Clark-3552',
 'Clark-3553'

640 Votes with hidden votes due to Nevada State Law requiring precincts with less than 10 votes to be denoted with an * asterisk to protect voter confidentiality. All counties with differences in the validation section of this report match the precincts with *'s above. 302 precincts have an election difference while 640 precincts have an asterisk. 

In [32]:
# #drop columns with asterisks for now
# nv_22['Votes'] = nv_22['Votes'].replace(to_replace=r'\*', value='0', regex=True)

# nv_22['Votes'] = nv_22['Votes'].astype(int)
# #filtered_20.to_csv('./filtered1.csv')

# Check Statewide Totals

In [33]:
pivoted = pivoted.fillna(0)

In [34]:
pivoted.loc[pivoted["G22ATGRCHA"]=="*"]

cand_col,UNIQUE_ID,Jurisdiction,Precinct,G22ATGRCHA,G22ATGDFOR,G22ATGOWRI,GCAJ1NFOR,GCAJ1NWRI,GCAJ1NWES,GCAJ2NGIB,...,GCON03DLEE,GCON04DHOR,GCON04RPET,G22USSDCOR,G22USSRLAX,G22USSILIN,G22USSOWRI,G22USSARUB,G22USSLSCO,COUNTYFP
24,Carson City-996,Carson City,996,*,*,*,*,*,*,*,...,0,0,0,*,*,*,*,*,*,510
46,Clark-1002,Clark,1002,*,*,*,*,*,*,*,...,*,0,0,*,*,*,*,*,*,003
47,Clark-1003,Clark,1003,*,*,*,*,*,*,*,...,0,0,0,*,*,*,*,*,*,003
57,Clark-1014,Clark,1014,*,*,*,*,*,*,*,...,*,0,0,*,*,*,*,*,*,003
89,Clark-1361,Clark,1361,*,*,*,*,*,*,*,...,0,0,0,*,*,*,*,*,*,003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,Washoe-SPARKS 9531,Washoe,SPARKS 9531,*,*,*,*,*,*,*,...,0,0,0,*,*,*,*,*,*,031
1677,Washoe-SPARKS 9532,Washoe,SPARKS 9532,*,*,*,*,*,*,*,...,0,0,0,*,*,*,*,*,*,031
1678,Washoe-SPARKS 9553,Washoe,SPARKS 9553,*,*,*,*,*,*,*,...,0,0,0,*,*,*,*,*,*,031
1679,Washoe-SPARKS 9558,Washoe,SPARKS 9558,*,*,*,*,*,*,*,...,0,0,0,*,*,*,*,*,*,031


In [35]:
for col in list(updated_dict.values()):
    print(col)
    print(list(pivoted[col].unique()))
    pivoted[col] = pivoted[col].astype(str).str.replace("*","0")
    pivoted[col] = pivoted[col].astype(str).str.replace("nan","0")
    pivoted[col] = pivoted[col].astype(int)
    



G22ATGRCHA
['263', '402', '442', '539', '555', '395', '596', '344', '412', '438', '372', '268', '320', '309', '419', '415', '507', '657', '308', '461', '368', '787', '445', '313', '*', '405', '193', '182', '107', '205', '407', '171', '252', '474', '466', '152', '272', '330', '339', '180', '504', '583', '210', '283', '188', '342', '314', '552', '409', '71', '38', '768', '534', '444', '962', '50', '103', '602', '658', '251', '350', '322', '562', '332', '337', '371', '437', '203', '593', '289', '294', '797', '396', '306', '526', '261', '394', '500', '315', '34', '312', '790', '1047', '635', '844', '377', '660', '1061', '923', '614', '354', '413', '851', '417', '425', '348', '458', '189', '447', '414', '347', '159', '220', '328', '689', '233', '465', '275', '1098', '579', '13', '809', '639', '597', '560', '343', '274', '487', '359', '686', '754', '511', '28', '815', '1126', '1095', '301', '501', '638', '122', '12', '260', '197', '684', '175', '18', '83', '366', '52', '63', '333', '170', '2

G22SOSRMAR
['292', '441', '487', '600', '652', '442', '657', '379', '446', '526', '449', '302', '382', '360', '485', '496', '569', '742', '335', '556', '889', '482', '363', '*', '214', '203', '130', '241', '466', '207', '282', '524', '537', '176', '297', '371', '381', '205', '596', '692', '354', '234', '301', '189', '369', '328', '558', '77', '39', '820', '576', '190', '461', '999', '366', '57', '121', '651', '714', '525', '505', '263', '362', '330', '609', '345', '376', '388', '468', '223', '626', '305', '313', '856', '402', '314', '536', '276', '414', '545', '398', '337', '35', '331', '851', '1119', '660', '924', '397', '710', '1157', '979', '658', '432', '425', '902', '444', '459', '465', '374', '172', '219', '342', '718', '247', '494', '1174', '14', '871', '719', '653', '349', '604', '373', '306', '531', '386', '747', '815', '28', '881', '1225', '1199', '334', '550', '682', '134', '12', '162', '288', '206', '766', '195', '18', '293', '87', '377', '329', '72', '343', '167', '230', '

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/2979910206.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pivoted[col] = pivoted[col].astype(str).str.replace("*","0")


GSL018DCON
[0, '181', '721', '237', '169', '395', '303', '582', '19', '*', '586', '795', '227', '802', '676', '337', '545', '559', '729', '895', '115', '210', '190', '196', '433', '321', '139']
GSL018RDEC
[0, '215', '446', '120', '109', '182', '266', '499', '28', '*', '381', '588', '147', '509', '342', '240', '464', '273', '641', '420', '80', '133', '100', '114', '282', '181', '43']
GSL019RYUR
[0, '*', '32', '1429', '931', '292', '1490', '757', '375', '1195', '1128', '35', '2116', '1760', '1084', '1151', '1120', '1376', '422', '1332', '1104', '895', '717', '17', '373', '1519', '635', '1819', '1163']
GSL020LLAR
[0, '26', '0', '2', '12', '17', '19', '45', '33', '21', '14', '7', '22', '11', '32', '9', '20', '28', '42', '23', '15']
GSL020DORE
[0, '363', '29', '30', '237', '474', '678', '1043', '790', '359', '193', '630', '274', '84', '550', '252', '667', '307', '306', '435', '258', '530', '771', '96', '338', '234']
GSL020RVAU
[0, '216', '13', '11', '162', '201', '476', '735', '581', '143',

['340', '487', '404', '482', '570', '448', '491', '360', '500', '489', '376', '282', '267', '310', '415', '414', '613', '473', '367', '486', '526', '601', '222', '*', '254', '106', '110', '192', '64', '128', '160', '70', '98', '145', '138', '60', '61', '89', '101', '162', '193', '99', '205', '343', '453', '431', '495', '714', '903', '425', '122', '71', '543', '740', '596', '932', '447', '35', '49', '523', '1048', '700', '611', '579', '478', '488', '477', '433', '569', '432', '355', '832', '231', '459', '753', '514', '454', '814', '396', '652', '894', '574', '506', '52', '348', '1031', '890', '956', '884', '180', '532', '766', '871', '705', '273', '650', '793', '745', '605', '665', '470', '324', '778', '628', '571', '421', '158', '697', '736', '629', '296', '1057', '4', '1040', '576', '747', '287', '607', '847', '286', '680', '210', '857', '1009', '496', '37', '895', '888', '1034', '297', '356', '281', '8', '258', '363', '770', '17', '400', '278', '854', '604', '36', '51', '776', '345',

In [36]:
official_statewide_totals = pd.read_csv("./raw-from-source/Nevada - nv_official_totals.csv")
official_statewide_totals["CAND_COL"] = official_statewide_totals["Contest"] + "-:-" + official_statewide_totals["Candidate"]
official_statewide_totals.drop(["Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Contest", "Candidate", "PERCENT", "TOTAL VOTES", "CALCULATED"],axis =1, inplace = True)

official_statewide_totals = official_statewide_totals[~official_statewide_totals["CAND_COL"].str.contains("Summary")]

df_transpose = official_statewide_totals.T

In [37]:
df_transpose.header = df_transpose.loc["CAND_COL",:]

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/2438063529.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_transpose.header = df_transpose.loc["CAND_COL",:]


In [38]:
df_transpose.columns = df_transpose.loc["CAND_COL",:]

In [39]:
df_transpose.reset_index(inplace = True, drop = False)

In [40]:
df_transpose = df_transpose[df_transpose["index"]!="CAND_COL"]

In [41]:
df_transpose.columns

Index(['index', 'United States Senator-:-CORTEZ MASTO, CATHERINE',
       'United States Senator-:-LAXALT, ADAM PAUL',
       'United States Senator-:-LINDEMANN, BARRY',
       'United States Senator-:-RUBINSON, BARRY',
       'United States Senator-:-SCOTT, NEIL',
       'United States Senator-:-NONE OF THESE CANDIDATES',
       'U.S. Representative in Congress, District 1-:-CAVANAUGH, KEN',
       'U.S. Representative in Congress, District 1-:-ROBERTSON, MARK',
       'U.S. Representative in Congress, District 1-:-TITUS, DINA',
       ...
       'Supreme Court Justice, Seat A-:-NONE OF THESE CANDIDATES',
       'Supreme Court Justice, Seat E-:-PARRAGUIRRE, RON D.',
       'Supreme Court Justice, Seat E-:-NONE OF THESE CANDIDATES',
       'Court of Appeals Judge, Department 1-:-FORSBERG, RHONDA K.',
       'Court of Appeals Judge, Department 1-:-WESTBROOK, DEBORAH',
       'Court of Appeals Judge, Department 1-:-NONE OF THESE CANDIDATES',
       'Court of Appeals Judge, Department 2-:

In [42]:
def get_election_type(contest):
    return "G"
#     first = contest.split("-:-")[0]
#     if first == 'Election':
#         e_type = "G"
#     elif first == "District" or first == "Dist":
#         e_type = "S"
#     elif first == 'Runoff':
#         e_type = "R"
#     elif first == 'Primary':
#         e_type = "P"
#     else:
#         print(contest)
#         raise ValueError
#     return e_type

def get_race(contest):
    mod_level = ""
    level = contest.split("-:-")[0]
    level_change_dict = {
        'Attorney General':'ATG',
        'Court of Appeals Judge':'CAJ',
        'Lieutenant Governor':'LTG',
        'Governor':'GOV',
        'State Controller':'CNT',
        'State Question':'SQ',
        'Secretary of State':'SOS',
        'State Assembly':'SL',
        'State Senate':'SU',
        'President':'PRE',
        'United States Senator':'USS',
        
        'State Treasurer':'TRE',
        'Supreme Court Justice':'SCJ',
        'U.S. Representative in Congress':'CON'}
    for val in level_change_dict.keys():
        if val in level:
            mod_level = level_change_dict[val]
            break
    if mod_level == "":
        print("NO CONTEST", contest)
    if mod_level in ["CON", "SU", "SL","CAJ","SCJ"]:
            dist_num = str(level.split(" ")[-1])
            if mod_level == "SL":
                fill = 3
            elif mod_level in ["CON","SU"]:
                fill = 2
            else:
                fill = 1
            mod_level += str(dist_num).zfill(fill)
    elif mod_level in ["SQ"]:
        mod_level += str(level.split(" ")[3])
        mod_level = "22"+mod_level
    else:
        mod_level = "22"+mod_level
    return mod_level

def get_party(contest):
    return "X"
    
def get_name(contest):
    candidate = contest.split("-:-")[1]
    if candidate == "NONE OF THESE CANDIDATES":
        return "WRI"
    candidate = candidate.upper()
    candidate = candidate.replace(" ", "")
    likely_last = candidate.split(" ")[-1]
    
    return likely_last[0:3]

In [43]:
keep_names  = ['index']

contest_name_change_dict_2 = {}

duplicate_value_list = []
for contest in df_transpose.columns:
    if contest not in keep_names:
        # Add in a condition about the 20
        value = get_election_type(contest) + get_race(contest)+ get_party(contest)+ get_name(contest)
        if value in contest_name_change_dict_2.values():
            duplicate_value_list.append(value)
        contest_name_change_dict_2[contest] = value 
#         else:
#             value = get_election_type(contest) + "22" + get_race(contest)+ get_party(contest)+ get_name(contest)
#             if value in contest_name_change_dict.values():
#                 duplicate_value_list.append(value)
#             contest_name_change_dict[contest] = value 

contest_name_change_dict_2["State Assembly, District 28-:-D'SILVA, REUBEN"] = "GSL028XDSI"
contest_name_change_dict_2['State Assembly, District 40-:-O\'NEILL, PHILIP "PK"'] = "GSL040XONE"

if len(duplicate_value_list) > 0:
    print("DUPLICATE VALUES")
    for val in [(k,v) for k, v in contest_name_change_dict_2.items() if v in duplicate_value_list]:
        print(val)

In [44]:
contest_name_change_dict_2

{'United States Senator-:-CORTEZ MASTO, CATHERINE': 'G22USSXCOR',
 'United States Senator-:-LAXALT, ADAM PAUL': 'G22USSXLAX',
 'United States Senator-:-LINDEMANN, BARRY': 'G22USSXLIN',
 'United States Senator-:-RUBINSON, BARRY': 'G22USSXRUB',
 'United States Senator-:-SCOTT, NEIL': 'G22USSXSCO',
 'United States Senator-:-NONE OF THESE CANDIDATES': 'G22USSXWRI',
 'U.S. Representative in Congress, District 1-:-CAVANAUGH, KEN': 'GCON01XCAV',
 'U.S. Representative in Congress, District 1-:-ROBERTSON, MARK': 'GCON01XROB',
 'U.S. Representative in Congress, District 1-:-TITUS, DINA': 'GCON01XTIT',
 'U.S. Representative in Congress, District 2-:-AMODEI, MARK  E.': 'GCON02XAMO',
 'U.S. Representative in Congress, District 2-:-BABER, DARRYL': 'GCON02XBAB',
 'U.S. Representative in Congress, District 2-:-BEST, RUSSELL': 'GCON02XBES',
 'U.S. Representative in Congress, District 2-:-KRAUSE, ELIZABETH MERCEDES': 'GCON02XKRA',
 'U.S. Representative in Congress, District 3-:-BECKER, APRIL': 'GCON03XB

In [45]:
df_transpose.rename(columns = contest_name_change_dict_2, inplace = True)
df_transpose.rename(columns = updated_dict, inplace = True)


/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/26200080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transpose.rename(columns = contest_name_change_dict_2, inplace = True)
/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/26200080.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transpose.rename(columns = updated_dict, inplace = True)


In [46]:
for col in list(updated_dict.values()):
    if col in list(df_transpose.columns):
        df_transpose[col] = df_transpose[col].astype(str).str.replace(",","")
        df_transpose[col] = df_transpose[col].astype(int)
    else:
        print(col)

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/3851364872.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transpose[col] = df_transpose[col].astype(str).str.replace(",","")
/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/3851364872.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transpose[col] = df_transpose[col].astype(int)


In [47]:
overlap = [i for i in df_transpose.columns if i in pivoted.columns]

In [48]:
df_transpose.rename(columns = {"index":"Jurisdiction"}, inplace= True)

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/2362110870.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transpose.rename(columns = {"index":"Jurisdiction"}, inplace= True)


In [49]:
df_transpose["Jurisdiction"] = df_transpose["Jurisdiction"].astype(str).str.title()

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/3218908867.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transpose["Jurisdiction"] = df_transpose["Jurisdiction"].astype(str).str.title()


In [50]:
pdv.county_totals_check(pivoted, "Precinct-Level", df_transpose, "Certified", overlap, "Jurisdiction", full_print=False, method='race')

***Countywide Totals Check***

G22USSDCOR contains differences in these counties:
	Clark has a difference of -70 votes


/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:35: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  diff = partner_df.groupby([county_col]).sum()[race]-source_df.groupby([county_col]).sum()[race]
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:44: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print("\t\t"+ partner_name + ": "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:44: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a

		Precinct-Level: 357205 votes
		Certified: 357275 votes
	Elko has a difference of -3 votes
		Precinct-Level: 3283 votes
		Certified: 3286 votes
	Eureka has a difference of -1 votes
		Precinct-Level: 72 votes
		Certified: 73 votes
	Mineral has a difference of -3 votes
		Precinct-Level: 657 votes
		Certified: 660 votes
	Nye has a difference of -1 votes
		Precinct-Level: 5956 votes
		Certified: 5957 votes
	Storey has a difference of -2 votes
		Precinct-Level: 719 votes
		Certified: 721 votes
	Washoe has a difference of -48 votes
		Precinct-Level: 98569 votes
		Certified: 98617 votes
G22USSRLAX contains differences in these counties:
	Clark has a difference of -109 votes
		Precinct-Level: 304024 votes
		Certified: 304133 votes
	Douglas has a difference of -4 votes
		Precinct-Level: 18886 votes
		Certified: 18890 votes
	Elko has a difference of -3 votes


/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:44: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print("\t\t"+ partner_name + ": "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")


		Precinct-Level: 12177 votes
		Certified: 12180 votes
	Esmeralda has a difference of -1 votes
		Precinct-Level: 343 votes
		Certified: 344 votes
	Eureka has a difference of -3 votes


...

/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:44: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print("\t\t"+ partner_name + ": "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:44: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print("\t\t"+ partner_name + ": "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:44: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated

		Certified: 5491 votes
	Storey has a difference of -1 votes
		Precinct-Level: 689 votes
		Certified: 690 votes
	Washoe has a difference of -46 votes
		Precinct-Level: 38269 votes
		Certified: 38315 votes
GCAJ2NGIB contains differences in these counties:
	Clark has a difference of -130 votes
		Precinct-Level: 438544 votes
		Certified: 438674 votes
	Douglas has a difference of -3 votes
		Precinct-Level: 19841 votes
		Certified: 19844 votes
	Elko has a difference of -5 votes
		Precinct-Level: 11380 votes
		Certified: 11385 votes
	Esmeralda has a difference of -10 votes
		Precinct-Level: 245 votes
		Certified: 255 votes
	Eureka has a difference of -4 votes
		Precinct-Level: 484 votes
		Certified: 488 votes
	Mineral has a difference of -1 votes
		Precinct-Level: 1210 votes
		Certified: 1211 votes
	Nye has a difference of -6 votes


/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:44: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print("\t\t"+ partner_name + ": "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:44: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print("\t\t"+ partner_name + ": "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:44: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated

		Precinct-Level: 13802 votes
		Certified: 13808 votes
	Storey has a difference of -7 votes
		Precinct-Level: 1689 votes
		Certified: 1696 votes
	Washoe has a difference of -105 votes
		Precinct-Level: 114548 votes
		Certified: 114653 votes
GCAJ2NWRI contains differences in these counties:
	Clark has a difference of -46 votes
		Precinct-Level: 154614 votes
		Certified: 154660 votes
	Elko has a difference of -1 votes
		Precinct-Level: 3006 votes
		Certified: 3007 votes
	Mineral has a difference of -3 votes
		Precinct-Level: 432 votes
		Certified: 435 votes
	Nye has a difference of -2 votes
		Precinct-Level: 4970 votes
		Certified: 4972 votes
	Storey has a difference of -1 votes
		Precinct-Level: 571 votes
		Certified: 572 votes
	Washoe has a difference of -39 votes
		Precinct-Level: 36788 votes
		Certified: 36827 votes
GCAJ3NBUL contains differences in these counties:
	Clark has a difference of -129 votes
		Precinct-Level: 433849 votes
		Certified: 433978 votes
	Douglas has a difference

/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:44: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print("\t\t"+ partner_name + ": "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:44: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print("\t\t"+ partner_name + ": "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:44: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated

In [51]:
pdv.county_totals_check(pivoted, "Precinct-Level", df_transpose, "Certified", overlap, "Jurisdiction", full_print=False, method='county')

/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:65: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  holder_1 = partner_df.groupby(county_col).sum()


***Countywide Totals Check***

Clark contains differences in these races:
	G22USSDCOR has a difference of -70 vote(s)
		Precinct-Level: 357205 vote(s)
		Certified: 357275 vote(s)
	G22USSRLAX has a difference of -109 vote(s)
		Precinct-Level: 304024 vote(s)
		Certified: 304133 vote(s)
	G22USSILIN has a difference of -2 vote(s)
		Precinct-Level: 4987 vote(s)
		Certified: 4989 vote(s)
	G22USSLSCO has a difference of -3 vote(s)
		Precinct-Level: 4038 vote(s)
		Certified: 4041 vote(s)
	G22USSOWRI has a difference of -1 vote(s)
		Precinct-Level: 8215 vote(s)
		Certified: 8216 vote(s)
	GCON01LCAV has a difference of -1 vote(s)
		Precinct-Level: 5533 vote(s)
		Certified: 5534 vote(s)
	GCON01RROB has a difference of -16 vote(s)
		Precinct-Level: 103099 vote(s)
		Certified: 103115 vote(s)
	GCON01DTIT has a difference of -11 vote(s)
		Precinct-Level: 115689 vote(s)
		Certified: 115700 vote(s)
	GCON03RBEC has a difference of -26 vote(s)
		Precinct-Level: 121057 vote(s)
		Certified: 121083 vote(s)


In [52]:
pivoted

cand_col,UNIQUE_ID,Jurisdiction,Precinct,G22ATGRCHA,G22ATGDFOR,G22ATGOWRI,GCAJ1NFOR,GCAJ1NWRI,GCAJ1NWES,GCAJ2NGIB,...,GCON03DLEE,GCON04DHOR,GCON04RPET,G22USSDCOR,G22USSRLAX,G22USSILIN,G22USSOWRI,G22USSARUB,G22USSLSCO,COUNTYFP
0,Carson City-101,Carson City,101,263,348,29,156,153,306,452,...,0,0,0,340,300,2,11,11,5,510
1,Carson City-103,Carson City,103,402,515,35,224,185,491,721,...,0,0,0,487,472,4,6,6,8,510
2,Carson City-105,Carson City,105,442,431,33,190,172,472,700,...,0,0,0,404,523,4,6,0,4,510
3,Carson City-107,Carson City,107,539,542,44,292,239,519,817,...,0,0,0,482,656,3,19,7,5,510
4,Carson City-109,Carson City,109,555,625,59,268,288,644,925,...,0,0,0,570,683,13,16,8,6,510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1686,White Pine-Precinct 5,White Pine,Precinct 5,716,144,36,189,290,404,650,...,0,0,0,145,757,9,11,7,13,033
1687,White Pine-Precinct 6,White Pine,Precinct 6,257,91,19,101,118,139,250,...,0,0,0,77,290,8,8,3,0,033
1688,White Pine-Precinct 7,White Pine,Precinct 7,92,29,2,30,29,62,91,...,0,0,0,24,100,1,1,3,1,033
1689,White Pine-Precinct 8,White Pine,Precinct 8,197,23,13,50,64,100,167,...,0,0,0,11,217,2,8,4,0,033


In [53]:
pivoted_grouped = pivoted.groupby("Jurisdiction").sum()

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/1792069660.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pivoted_grouped = pivoted.groupby("Jurisdiction").sum()


In [54]:
pivoted_grouped.reset_index(inplace = True)

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/1549521104.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pivoted_grouped.reset_index(inplace = True)


In [55]:
pivoted_grouped

cand_col,Jurisdiction,G22ATGRCHA,G22ATGDFOR,G22ATGOWRI,GCAJ1NFOR,GCAJ1NWRI,GCAJ1NWES,GCAJ2NGIB,GCAJ2NWRI,GCAJ3NBUL,...,GCON03RBEC,GCON03DLEE,GCON04DHOR,GCON04RPET,G22USSDCOR,G22USSRLAX,G22USSILIN,G22USSOWRI,G22USSARUB,G22USSLSCO
0,Carson City,10335,11164,1006,5400,5226,11069,16837,4751,15818,...,0,0,0,0,10337,12451,234,308,189,175
1,Churchill,5839,2874,558,2176,2304,4392,6799,1968,6611,...,0,0,0,0,2423,7032,110,153,63,99
2,Clark,274888,361929,19380,152348,138267,315085,438544,154614,433849,...,121057,131078,109171,87920,357205,304024,4987,8215,3144,4038
3,Douglas,16430,10276,1227,6059,6550,11740,19841,4639,17870,...,0,0,0,0,9466,18886,214,288,138,173
4,Elko,11502,3286,570,4317,3539,6756,11380,3006,10941,...,0,0,0,0,3283,12177,204,219,153,149
5,Esmeralda,295,74,33,77,167,153,245,135,233,...,0,0,73,363,68,343,13,22,5,2
6,Eureka,601,86,50,168,258,279,484,210,441,...,0,0,0,0,72,668,8,17,10,2
7,Humboldt,3754,1600,370,1334,1644,2601,4259,1267,4084,...,0,0,0,0,1358,4456,75,132,46,45
8,Lander,1462,477,121,511,581,971,1589,451,1537,...,0,0,0,0,387,1673,42,58,29,23
9,Lincoln,1536,375,138,538,601,801,1490,442,1408,...,0,0,334,1799,321,1727,12,62,14,21


In [56]:
df_transpose

CAND_COL,Jurisdiction,G22USSDCOR,G22USSRLAX,G22USSILIN,G22USSARUB,G22USSLSCO,G22USSOWRI,GCON01LCAV,GCON01RROB,GCON01DTIT,...,GSCJANWRI,GSCJENPAR,GSCJENWRI,GCAJ1NFOR,GCAJ1NWES,GCAJ1NWRI,GCAJ2NGIB,GCAJ2NWRI,GCAJ3NBUL,GCAJ3NWRI
0,Carson City,10337,12451,234,189,175,308,0,0,0,...,5081,16805,4872,5400,11069,5226,16837,4751,15818,5613
1,Churchill,2423,7032,110,63,99,153,0,0,0,...,1990,6845,1951,2176,4392,2304,6799,1968,6611,2164
2,Clark,357275,304133,4989,3144,4041,8216,5534,103115,115700,...,146528,437403,157020,152406,315161,138302,438674,154660,433978,159557
3,Douglas,9466,18890,214,138,173,288,0,0,0,...,5322,18770,5281,6060,11740,6551,19844,4639,17871,5745
4,Elko,3286,12180,204,153,149,219,0,0,0,...,3234,11239,3150,4318,6760,3540,11385,3007,10947,3417
5,Esmeralda,68,344,13,5,2,22,0,0,0,...,129,268,131,78,161,168,255,135,243,147
6,Eureka,73,671,8,10,2,17,0,0,0,...,210,493,208,171,280,258,488,210,445,251
7,Humboldt,1358,4456,75,46,45,132,0,0,0,...,1235,4248,1273,1334,2601,1644,4259,1267,4084,1382
8,Lander,387,1673,42,29,23,58,0,0,0,...,452,1608,439,511,971,581,1589,451,1537,503
9,Lincoln,321,1727,12,14,21,62,0,0,0,...,429,1472,466,538,801,601,1490,442,1408,522


In [57]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)


In [58]:
pivoted_astericks

cand_col,UNIQUE_ID,Jurisdiction,Precinct,G22ATGRCHA,G22ATGDFOR,G22ATGOWRI,GCAJ1NFOR,GCAJ1NWRI,GCAJ1NWES,GCAJ2NGIB,...,GCON03DLEE,GCON04DHOR,GCON04RPET,G22USSDCOR,G22USSRLAX,G22USSILIN,G22USSOWRI,G22USSARUB,G22USSLSCO,COUNTYFP
0,Carson City-101,Carson City,101,263,348,29,156,153,306,452,...,0,0,0,340,300,2,11,11,5,510
1,Carson City-103,Carson City,103,402,515,35,224,185,491,721,...,0,0,0,487,472,4,6,6,8,510
2,Carson City-105,Carson City,105,442,431,33,190,172,472,700,...,0,0,0,404,523,4,6,0,4,510
3,Carson City-107,Carson City,107,539,542,44,292,239,519,817,...,0,0,0,482,656,3,19,7,5,510
4,Carson City-109,Carson City,109,555,625,59,268,288,644,925,...,0,0,0,570,683,13,16,8,6,510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1686,White Pine-Precinct 5,White Pine,Precinct 5,716,144,36,189,290,404,650,...,0,0,0,145,757,9,11,7,13,033
1687,White Pine-Precinct 6,White Pine,Precinct 6,257,91,19,101,118,139,250,...,0,0,0,77,290,8,8,3,0,033
1688,White Pine-Precinct 7,White Pine,Precinct 7,92,29,2,30,29,62,91,...,0,0,0,24,100,1,1,3,1,033
1689,White Pine-Precinct 8,White Pine,Precinct 8,197,23,13,50,64,100,167,...,0,0,0,11,217,2,8,4,0,033


In [59]:
df_asterisks = df_transpose.copy(deep = True)

In [60]:
for val in df_asterisks.columns:
    if val != "Jurisdiction":
        df_asterisks.loc[:,val] = 0

In [61]:
df_asterisks.to_csv("./votes_adding_back.csv", index = False)

In [62]:
df_asterisks

CAND_COL,Jurisdiction,G22USSDCOR,G22USSRLAX,G22USSILIN,G22USSARUB,G22USSLSCO,G22USSOWRI,GCON01LCAV,GCON01RROB,GCON01DTIT,...,GSCJANWRI,GSCJENPAR,GSCJENWRI,GCAJ1NFOR,GCAJ1NWES,GCAJ1NWRI,GCAJ2NGIB,GCAJ2NWRI,GCAJ3NBUL,GCAJ3NWRI
0,Carson City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Churchill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Clark,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Douglas,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Elko,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Esmeralda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Eureka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Humboldt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Lander,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Lincoln,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
clear_list = []
for (idx, row) in df_transpose.iterrows():
    county = row['Jurisdiction']
    print(county)
    for col in list(updated_dict.values()):
        
        precinct_value = pivoted_grouped.loc[pivoted_grouped["Jurisdiction"]==county, col].values[0]
        official_value = row.loc[col]
        diff = official_value - precinct_value 
        if diff != 0:
            print(col)
            print(diff)
            df_asterisks.loc[df_asterisks["Jurisdiction"]==county, col] = diff
        elif diff == 0:
            clear_list.append((county, col))

Carson City
Churchill
Clark
G22ATGRCHA
98
G22ATGDFOR
76
G22ATGOWRI
7
GCAJ1NFOR
58
GCAJ1NWRI
35
GCAJ1NWES
76
GCAJ2NGIB
130
GCAJ2NWRI
46
GCAJ3NBUL
129
GCAJ3NWRI
47
G22GOVABRI
2
G22GOVLDAV
3
G22GOVRLOM
111
G22GOVOWRI
1
G22GOVDSIS
67
G22LTGRANT
112
G22LTGDCAN
61
G22LTGIDEL
1
G22LTGAHOG
1
G22LTGOWRI
3
G22LTGLTAC
6
G22SQ1NO
89
G22SQ1YES
97
G22SQ2NO
98
G22SQ2YES
87
G22SQ3NO
85
G22SQ3YES
100
G22SOSDAGU
71
G22SOSLCRA
4
G22SOSRMAR
107
G22SOSOWRI
3
GSL003RLEM
10
GSL003DTOR
16
GSL004LBUR
11
GSL004RMCA
28
GSL006RRIO
3
GSL006DSUM
1
GSL010RHER
1
GSL010DNGU
1
GSL011DDUR
1
GSL013RHIB
7
GSL015RBAN
5
GSL015DWAT
2
GSL016DGON
2
GSL016RHOL
4
GSL017RPAW
10
GSL017DTHO
6
GSL018DCON
1
GSL018RDEC
1
GSL019RYUR
7
GSL021DMAR
1
GSL021RPET
1
GSL023DBRI
4
GSL023RGAL
7
GSL036RHAF
20
GSL037DBAC
5
GSL037RDEA
11
GSL041RBOD
1
GSL041DJAU
2
G22CNTRMAT
110
G22CNTOWRI
3
G22CNTLPRO
6
G22CNTDSPI
63
GSU10DDON
4
GSU10RGRA
9
GSU12RARR
2
GSU12DPAZ
3
GSU02DFLO
1
GSU20DFOU
6
GSU20RSTO
14
G22TREDCON
69
G22TRELELL
6
G22TRERFIO
97
G22TRE

In [64]:
for col in df_asterisks:
    print(min(df_asterisks[col]))
    
print(df_asterisks["GSU14RHAN"])

Carson City
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0      0
1      0
2      0
3      0
4      0
5      0
6      3
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15    62
16     0
Name: GSU14RHAN, dtype: int64


In [65]:
for val in clear_list:
    if pivoted_astericks[(pivoted_astericks["Jurisdiction"]==val[0])&(pivoted_astericks[val[1]]=="*")].shape[0] > 0:
        print("REMOVING")
        print(val)

    pivoted_astericks.loc[pivoted_astericks["Jurisdiction"]==val[0],val[1]] = pivoted_astericks.loc[pivoted_astericks["Jurisdiction"]==val[0],val[1]].astype(str).str.replace("*","0")
    pivoted_astericks.loc[pivoted_astericks["Jurisdiction"]==val[0],val[1]] = pivoted_astericks.loc[pivoted_astericks["Jurisdiction"]==val[0],val[1]].astype(int)

REMOVING
('Carson City', 'G22ATGRCHA')
REMOVING
('Carson City', 'G22ATGDFOR')
REMOVING
('Carson City', 'G22ATGOWRI')
REMOVING
('Carson City', 'GCAJ1NFOR')
REMOVING
('Carson City', 'GCAJ1NWRI')
REMOVING
('Carson City', 'GCAJ1NWES')
REMOVING
('Carson City', 'GCAJ2NGIB')
REMOVING
('Carson City', 'GCAJ2NWRI')
REMOVING
('Carson City', 'GCAJ3NBUL')
REMOVING
('Carson City', 'GCAJ3NWRI')
REMOVING
('Carson City', 'G22GOVABRI')
REMOVING
('Carson City', 'G22GOVLDAV')
REMOVING
('Carson City', 'G22GOVRLOM')
REMOVING
('Carson City', 'G22GOVOWRI')
REMOVING
('Carson City', 'G22GOVDSIS')
REMOVING
('Carson City', 'G22LTGRANT')
REMOVING
('Carson City', 'G22LTGDCAN')
REMOVING
('Carson City', 'G22LTGIDEL')
REMOVING
('Carson City', 'G22LTGAHOG')
REMOVING
('Carson City', 'G22LTGOWRI')
REMOVING
('Carson City', 'G22LTGLTAC')
REMOVING
('Carson City', 'G22SQ1NO')
REMOVING
('Carson City', 'G22SQ1YES')
REMOVING
('Carson City', 'G22SQ2NO')
REMOVING
('Carson City', 'G22SQ2YES')
REMOVING
('Carson City', 'G22SQ3NO')
R

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/2107864279.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pivoted_astericks.loc[pivoted_astericks["Jurisdiction"]==val[0],val[1]] = pivoted_astericks.loc[pivoted_astericks["Jurisdiction"]==val[0],val[1]].astype(str).str.replace("*","0")


REMOVING
('Carson City', 'G22TREDCON')
REMOVING
('Carson City', 'G22TRELELL')
REMOVING
('Carson City', 'G22TRERFIO')
REMOVING
('Carson City', 'G22TREAHEN')
REMOVING
('Carson City', 'G22TREOWRI')
REMOVING
('Carson City', 'GSCJANBEL')
REMOVING
('Carson City', 'GSCJANWRI')
REMOVING
('Carson City', 'GSCJENWRI')
REMOVING
('Carson City', 'GSCJENPAR')
REMOVING
('Carson City', 'G22USSDCOR')
REMOVING
('Carson City', 'G22USSRLAX')
REMOVING
('Carson City', 'G22USSILIN')
REMOVING
('Carson City', 'G22USSOWRI')
REMOVING
('Carson City', 'G22USSARUB')
REMOVING
('Carson City', 'G22USSLSCO')
REMOVING
('Churchill', 'GCON04DHOR')
REMOVING
('Churchill', 'GCON04RPET')
REMOVING
('Clark', 'G22SOSAHAN')
REMOVING
('Clark', 'GSL008RLOG')
REMOVING
('Clark', 'GSL008DNGU')
REMOVING
('Clark', 'GSL011RKRA')
REMOVING
('Clark', 'GSL012DCAR')
REMOVING
('Clark', 'GSL012RLAR')
REMOVING
('Clark', 'GSL013DRUC')
REMOVING
('Clark', 'GSL014DMOS')
REMOVING
('Clark', 'GSL014RSTA')
REMOVING
('Clark', 'GSL023LMAN')
REMOVING
('Clar

REMOVING
('Storey', 'G22GOVLDAV')
REMOVING
('Storey', 'G22GOVOWRI')
REMOVING
('Storey', 'G22GOVDSIS')
REMOVING
('Storey', 'G22LTGDCAN')
REMOVING
('Storey', 'G22LTGIDEL')
REMOVING
('Storey', 'G22LTGAHOG')
REMOVING
('Storey', 'G22LTGOWRI')
REMOVING
('Storey', 'G22LTGLTAC')
REMOVING
('Storey', 'G22SOSLCRA')
REMOVING
('Storey', 'G22SOSOWRI')
REMOVING
('Storey', 'GSL040DMCD')
REMOVING
('Storey', 'GSL040LTOL')
REMOVING
('Storey', 'G22CNTOWRI')
REMOVING
('Storey', 'G22CNTLPRO')
REMOVING
('Storey', 'G22CNTDSPI')
REMOVING
('Storey', 'GSU16DSIM')
REMOVING
('Storey', 'G22TRELELL')
REMOVING
('Storey', 'G22TREAHEN')
REMOVING
('Storey', 'G22TREOWRI')
REMOVING
('Storey', 'GCON02LBAB')
REMOVING
('Storey', 'GCON02ABES')
REMOVING
('Storey', 'GCON02DKRA')
REMOVING
('Storey', 'G22USSILIN')
REMOVING
('Storey', 'G22USSOWRI')
REMOVING
('Storey', 'G22USSARUB')
REMOVING
('Storey', 'G22USSLSCO')
REMOVING
('Washoe', 'G22SOSOWRI')
REMOVING
('Washoe', 'GSL040DMCD')
REMOVING
('Washoe', 'GSL040RONE')
REMOVING
('Wash

In [66]:
more_help = [] 
asterisk_list = []

for index,row in pivoted_astericks.iterrows():
    any_value = False
    any_asterisk = False
    for col in list(updated_dict.values()):
#         print(row["UNIQUE_ID"])
#         print(row[col])
#         print(col)
        if row[col] == "*":
            any_asterisk = True
        elif int(row[col]) > 0:
            any_value = True
    if any_asterisk and any_value:
        more_help.append(row["UNIQUE_ID"])
    elif any_asterisk:
        asterisk_list.append(row["UNIQUE_ID"])

In [67]:
asterisk_list

['Clark-1002',
 'Clark-1003',
 'Clark-1014',
 'Clark-1361',
 'Clark-1363',
 'Clark-1553',
 'Clark-1665',
 'Clark-1732',
 'Clark-2000',
 'Clark-2012',
 'Clark-2017',
 'Clark-2020',
 'Clark-2042',
 'Clark-2057',
 'Clark-2059',
 'Clark-2060',
 'Clark-2061',
 'Clark-2064',
 'Clark-2068',
 'Clark-2069',
 'Clark-2070',
 'Clark-2075',
 'Clark-2078',
 'Clark-2356',
 'Clark-2468',
 'Clark-2478',
 'Clark-2486',
 'Clark-2487',
 'Clark-2488',
 'Clark-2489',
 'Clark-2503',
 'Clark-2608',
 'Clark-2612',
 'Clark-2623',
 'Clark-2625',
 'Clark-2641',
 'Clark-2681',
 'Clark-2683',
 'Clark-2685',
 'Clark-2694',
 'Clark-2703',
 'Clark-2706',
 'Clark-2723',
 'Clark-2724',
 'Clark-2742',
 'Clark-3004',
 'Clark-3005',
 'Clark-3046',
 'Clark-3047',
 'Clark-3051',
 'Clark-3053',
 'Clark-3054',
 'Clark-3070',
 'Clark-3151',
 'Clark-3191',
 'Clark-3433',
 'Clark-3439',
 'Clark-3457',
 'Clark-3537',
 'Clark-3545',
 'Clark-3552',
 'Clark-3553',
 'Clark-3559',
 'Clark-3562',
 'Clark-3571',
 'Clark-3574',
 'Clark-35

In [68]:
more_help

['Clark-3725',
 'Clark-4026',
 'Esmeralda-Precinct 4',
 'Washoe-RENO-VERDI 3021',
 'Washoe-RENO-VERDI 5032',
 'Washoe-RENO-VERDI 5053',
 'Washoe-RENO-VERDI 7307',
 'Washoe-RENO-VERDI 7556',
 'Washoe-RENO-VERDI 8101',
 'Washoe-RENO-VERDI 8118',
 'Washoe-RENO-VERDI 8258']

In [69]:
full_holder = []

for col in list(updated_dict.values()):
    if sum(df_asterisks[col]) > 0:
        df_asterisks_allocate = df_asterisks[df_asterisks[col]>0][["Jurisdiction",col]].copy(deep = True)
        holder = pivoted_astericks[pivoted_astericks[col]=="*"].copy(deep = True)
        holder[col] = holder[col].astype(str).str.replace("*","0")
        holder[col] = holder[col].astype(int)
        holder = holder[["UNIQUE_ID","Jurisdiction",col]]
        temp = pdv.allocate_absentee(holder,df_asterisks_allocate,[col],"Jurisdiction",True)
        for index, row in temp.iterrows():
            full_holder.append((row["UNIQUE_ID"], row[col], col))

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/2039637153.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  holder[col] = holder[col].astype(str).str.replace("*","0")
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:270: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  precinct_specific_totals = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:352: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric

Special Allocation Needed [['Clark', 'G22ATGRCHA'], ['Douglas', 'G22ATGRCHA'], ['Elko', 'G22ATGRCHA'], ['Esmeralda', 'G22ATGRCHA'], ['Eureka', 'G22ATGRCHA'], ['Mineral', 'G22ATGRCHA'], ['Nye', 'G22ATGRCHA'], ['Storey', 'G22ATGRCHA'], ['Washoe', 'G22ATGRCHA']]


/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/2039637153.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  holder[col] = holder[col].astype(str).str.replace("*","0")
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:270: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  precinct_specific_totals = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:352: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric

Special Allocation Needed [['Clark', 'G22ATGDFOR'], ['Elko', 'G22ATGDFOR'], ['Esmeralda', 'G22ATGDFOR'], ['Eureka', 'G22ATGDFOR'], ['Mineral', 'G22ATGDFOR'], ['Nye', 'G22ATGDFOR'], ['Storey', 'G22ATGDFOR'], ['Washoe', 'G22ATGDFOR']]


/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:374: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_receiving_votes.loc[:,race]+=df_receiving_votes.loc[:,add_var]
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:375: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_receiving_votes.loc[:,race] = df_receiving_votes.loc[:,race].astype(int)
/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/2039637153.py:7: FutureWarning: The default value of regex will change from True to False in a futu

...

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/2039637153.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  holder[col] = holder[col].astype(str).str.replace("*","0")
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:270: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  precinct_specific_totals = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:352: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric

Special Allocation Needed [['Clark', 'G22USSOWRI'], ['Washoe', 'G22USSOWRI']]


/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:270: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  precinct_specific_totals = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:352: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  first_allocation = pd.DataFrame(df_receiving_votes.groupby([col_allocating]).sum())
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:374: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of

Special Allocation Needed [['Washoe', 'G22USSARUB']]
Special Allocation Needed [['Clark', 'G22USSLSCO'], ['Washoe', 'G22USSLSCO']]


/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:374: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_receiving_votes.loc[:,race]+=df_receiving_votes.loc[:,add_var]
/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:375: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_receiving_votes.loc[:,race] = df_receiving_votes.loc[:,race].astype(int)


In [70]:
len(full_holder)

20446

In [71]:
for val in full_holder:
    #print(val)
    pivoted_astericks.loc[pivoted_astericks["UNIQUE_ID"]==val[0],val[2]] = int(val[1])

In [72]:
for val in list(updated_dict.values()):
    pivoted_astericks[val] = pivoted_astericks[val].astype(int)

In [73]:
pivoted_astericks.groupby("Jurisdiction").sum()

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_89126/2489501638.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pivoted_astericks.groupby("Jurisdiction").sum()


cand_col,G22ATGRCHA,G22ATGDFOR,G22ATGOWRI,GCAJ1NFOR,GCAJ1NWRI,GCAJ1NWES,GCAJ2NGIB,GCAJ2NWRI,GCAJ3NBUL,GCAJ3NWRI,...,GCON03RBEC,GCON03DLEE,GCON04DHOR,GCON04RPET,G22USSDCOR,G22USSRLAX,G22USSILIN,G22USSOWRI,G22USSARUB,G22USSLSCO
Jurisdiction,,,,,,,,,,,,,,,,,,,,,
Carson City,10335,11164,1006,5400,5226,11069,16837,4751,15818,5613,...,0,0,0,0,10337,12451,234,308,189,175
Churchill,5839,2874,558,2176,2304,4392,6799,1968,6611,2164,...,0,0,0,0,2423,7032,110,153,63,99
Clark,274986,362005,19387,152406,138302,315161,438674,154660,433978,159557,...,121083,131086,109220,87988,357275,304133,4989,8216,3144,4041
Douglas,16432,10276,1228,6060,6551,11740,19844,4639,17871,5745,...,0,0,0,0,9466,18890,214,288,138,173
Elko,11506,3288,570,4318,3540,6760,11385,3007,10947,3417,...,0,0,0,0,3286,12180,204,219,153,149
Esmeralda,302,77,33,78,168,161,255,135,243,147,...,0,0,73,364,68,344,13,22,5,2
Eureka,604,87,50,171,258,280,488,210,445,251,...,0,0,0,0,73,671,8,17,10,2
Humboldt,3754,1600,370,1334,1644,2601,4259,1267,4084,1382,...,0,0,0,0,1358,4456,75,132,46,45
Lander,1462,477,121,511,581,971,1589,451,1537,503,...,0,0,0,0,387,1673,42,58,29,23


In [74]:
df_transpose

CAND_COL,Jurisdiction,G22USSDCOR,G22USSRLAX,G22USSILIN,G22USSARUB,G22USSLSCO,G22USSOWRI,GCON01LCAV,GCON01RROB,GCON01DTIT,...,GSCJANWRI,GSCJENPAR,GSCJENWRI,GCAJ1NFOR,GCAJ1NWES,GCAJ1NWRI,GCAJ2NGIB,GCAJ2NWRI,GCAJ3NBUL,GCAJ3NWRI
0,Carson City,10337,12451,234,189,175,308,0,0,0,...,5081,16805,4872,5400,11069,5226,16837,4751,15818,5613
1,Churchill,2423,7032,110,63,99,153,0,0,0,...,1990,6845,1951,2176,4392,2304,6799,1968,6611,2164
2,Clark,357275,304133,4989,3144,4041,8216,5534,103115,115700,...,146528,437403,157020,152406,315161,138302,438674,154660,433978,159557
3,Douglas,9466,18890,214,138,173,288,0,0,0,...,5322,18770,5281,6060,11740,6551,19844,4639,17871,5745
4,Elko,3286,12180,204,153,149,219,0,0,0,...,3234,11239,3150,4318,6760,3540,11385,3007,10947,3417
5,Esmeralda,68,344,13,5,2,22,0,0,0,...,129,268,131,78,161,168,255,135,243,147
6,Eureka,73,671,8,10,2,17,0,0,0,...,210,493,208,171,280,258,488,210,445,251
7,Humboldt,1358,4456,75,46,45,132,0,0,0,...,1235,4248,1273,1334,2601,1644,4259,1267,4084,1382
8,Lander,387,1673,42,29,23,58,0,0,0,...,452,1608,439,511,971,581,1589,451,1537,503
9,Lincoln,321,1727,12,14,21,62,0,0,0,...,429,1472,466,538,801,601,1490,442,1408,522


In [75]:
pdv.county_totals_check(pivoted_astericks, "Precinct-Level", df_transpose, "Certified", overlap, "Jurisdiction", full_print=False, method='county')

***Countywide Totals Check***

Counties that match:

['Carson City', 'Churchill', 'Clark', 'Douglas', 'Elko', 'Esmeralda', 'Eureka', 'Humboldt', 'Lander', 'Lincoln', 'Lyon', 'Mineral', 'Nye', 'Pershing', 'Storey', 'Washoe', 'White Pine']


/Users/peterhorton/Documents/RDH/pber_collection/NV/2022/pdv_functions.py:65: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  holder_1 = partner_df.groupby(county_col).sum()


In [76]:
pivoted_astericks_unmod[(pivoted_astericks_unmod["Jurisdiction"]=="Lander")]["GSU14RHAN"]

1065    298
1066    329
1067    132
1068    518
1069    514
1070     54
1071     71
1072     11
1073     56
Name: GSU14RHAN, dtype: object

# Add a COUNTYFP

In [77]:
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")
fips_file = fips_file[fips_file["State"]=="Nevada"]
fips_file["FIPS County"] = fips_file["FIPS County"].astype(str)
fips_file["FIPS County"] = fips_file["FIPS County"].str.zfill(3)
fips_dict = dict(zip(fips_file['County Name'], fips_file['FIPS County']))
pivoted_astericks['COUNTYFP'] = pivoted_astericks['COUNTYFP'].map(fips_dict).fillna(pivoted_astericks['Jurisdiction'])
pivoted_astericks['COUNTYFP'] = pivoted_astericks['COUNTYFP'].astype(str).str.zfill(3)

# Print statements to check the county FIPs we've added
print(pivoted_astericks['COUNTYFP'].unique())

['Carson City' 'Churchill' 'Clark' 'Douglas' 'Elko' 'Esmeralda' 'Eureka'
 'Humboldt' 'Lander' 'Lincoln' 'Lyon' 'Mineral' 'Nye' 'Pershing' 'Storey'
 'Washoe' 'White Pine']


# Final Clean and Export

In [78]:
pivoted_astericks

cand_col,UNIQUE_ID,Jurisdiction,Precinct,G22ATGRCHA,G22ATGDFOR,G22ATGOWRI,GCAJ1NFOR,GCAJ1NWRI,GCAJ1NWES,GCAJ2NGIB,...,GCON03DLEE,GCON04DHOR,GCON04RPET,G22USSDCOR,G22USSRLAX,G22USSILIN,G22USSOWRI,G22USSARUB,G22USSLSCO,COUNTYFP
0,Carson City-101,Carson City,101,263,348,29,156,153,306,452,...,0,0,0,340,300,2,11,11,5,Carson City
1,Carson City-103,Carson City,103,402,515,35,224,185,491,721,...,0,0,0,487,472,4,6,6,8,Carson City
2,Carson City-105,Carson City,105,442,431,33,190,172,472,700,...,0,0,0,404,523,4,6,0,4,Carson City
3,Carson City-107,Carson City,107,539,542,44,292,239,519,817,...,0,0,0,482,656,3,19,7,5,Carson City
4,Carson City-109,Carson City,109,555,625,59,268,288,644,925,...,0,0,0,570,683,13,16,8,6,Carson City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1686,White Pine-Precinct 5,White Pine,Precinct 5,716,144,36,189,290,404,650,...,0,0,0,145,757,9,11,7,13,White Pine
1687,White Pine-Precinct 6,White Pine,Precinct 6,257,91,19,101,118,139,250,...,0,0,0,77,290,8,8,3,0,White Pine
1688,White Pine-Precinct 7,White Pine,Precinct 7,92,29,2,30,29,62,91,...,0,0,0,24,100,1,1,3,1,White Pine
1689,White Pine-Precinct 8,White Pine,Precinct 8,197,23,13,50,64,100,167,...,0,0,0,11,217,2,8,4,0,White Pine


In [79]:
pivoted_astericks.rename(columns = {'Jurisdiction':'CNTY_NAME',
                              'Precinct':'PREC'}, inplace = True)

In [80]:
pivoted_astericks = pivoted_astericks[["UNIQUE_ID","COUNTYFP","CNTY_NAME","PREC"]+list(sorted_updated_dict.values())]

In [81]:
if not os.path.exists("./nv_2022_gen_prec"):
    os.mkdir("./nv_2022_gen_prec")
    
pivoted_astericks.to_csv("./nv_2022_gen_prec/nv_2022_gen_prec.csv", index = False)